In [1]:
import pandas as pd
from pricer.pricer import Pricer
from pricer.pricer_factory import PricerFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing pricer_class
pricer_class = PricerFactory.build(Pricer.WEEKLY_STOCK_WINDOW)

In [3]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [6]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = pricer_class.training_set(ticker,prices)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:33<00:00, 14.49it/s]


In [7]:
data = pd.concat(training_sets)

In [8]:
training_data = data.dropna().copy().sort_values(["year","week"])

In [9]:
training_data = training_data.groupby(["year","quarter","week","ticker"]).mean().reset_index()

In [10]:
# pricer_class.db.connect()
# relevant_columns = list(set(list(training_data.columns)) - set(pricer_class.factors))
# pricer_class.db.store("sim",training_data[relevant_columns])
# pricer_class.db.disconnect()

In [10]:
pricer_class.db.connect()
pricer_class.db.drop("predictions")
relevant_columns = list(set(list(training_data.columns)) - set(pricer_class.factors))
predictions = pricer_class.sim_processor(training_data)
pricer_class.db.store("predictions",predictions[predictions["year"]==current_year][relevant_columns])
pricer_class.db.disconnect()